In [1]:
'''!python -m spacy download en_core_web_trf

!pip install paddlepaddle-gpu==2.3.0.post110 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
#!python3 -m pip install paddlepaddle-gpu==2.0.2 -f https://paddlepaddle.org.cn/whl/stable/noavx.html
!pip install pdf2image
!apt-get update
!apt-get install poppler-utils

######Dont install#!python3 -m pip install paddlepaddle-gpu
!python -m pip install --force-reinstall paddlepaddle==2.5

!pip install "paddleocr>=2.0.1"
#!pip install protobuf==3.20.0
!git clone https://github.com/PaddlePaddle/PaddleOCR.git

!wget https://paddleocr.bj.bcebos.com/whl/layoutparser-0.0.0-py3-none-any.whl
!pip install -U layoutparser-0.0.0-py3-none-any.whl

### Install first one if code doea not work install others
!pip install protobuf==3.20.*
#!pip install -U layoutparser-0.0.0-py3-none-any.whl
#!wget https://www.openssl.org/source/openssl-1.1.1o.tar.gz
#!wget http://nz2.archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2_amd64.deb
#!sudo dpkg -i libssl1.1_1.1.1f-1ubuntu2_amd64.deb

!wget https://www.openssl.org/source/openssl-1.1.1o.tar.gz
!wget http://nz2.archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2_amd64.deb
!sudo dpkg -i libssl1.1_1.1.1f-1ubuntu2_amd64.deb'''

'!python -m spacy download en_core_web_trf\n\n!pip install paddlepaddle-gpu==2.3.0.post110 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html\n#!python3 -m pip install paddlepaddle-gpu==2.0.2 -f https://paddlepaddle.org.cn/whl/stable/noavx.html\n!pip install pdf2image\n!apt-get update\n!apt-get install poppler-utils\n\n######Dont install#!python3 -m pip install paddlepaddle-gpu\n!python -m pip install --force-reinstall paddlepaddle==2.5\n\n!pip install "paddleocr>=2.0.1"\n#!pip install protobuf==3.20.0\n!git clone https://github.com/PaddlePaddle/PaddleOCR.git\n\n!wget https://paddleocr.bj.bcebos.com/whl/layoutparser-0.0.0-py3-none-any.whl\n!pip install -U layoutparser-0.0.0-py3-none-any.whl\n\n### Install first one if code doea not work install others\n!pip install protobuf==3.20.*\n#!pip install -U layoutparser-0.0.0-py3-none-any.whl\n#!wget https://www.openssl.org/source/openssl-1.1.1o.tar.gz\n#!wget http://nz2.archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1

In [2]:
from pdf2image import convert_from_path
from PIL import Image
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import spacy
import spacy_transformers

import cv2
import layoutparser as lp

from paddleocr import PaddleOCR, draw_ocr

import re
from datetime import datetime

import os
import glob
from pathlib import Path

In [3]:
# Function to check if a string matches the GSTIN pattern
def is_valid_gstin(gstin):
  # Define the GSTIN pattern
  gstin_pattern = r'^[0-9]{2}[A-Z]{5}[0-9]{4}[A-Z]{1}[0-9]{1}[A-Z]{1}[A-Z0-9]{1}$'
  return bool(re.match(gstin_pattern, gstin))

# Function to check if a string matches any of the specified date formats
def is_valid_date_time(date_time_str):
  # Define a list of date formats to check
  date_formats = ["%d.%m.%y%H:%M:%S", "%Y-%m-%d %H:%M:%S", "%Y-%m-%d", "%d/%m/%Y", "%m/%d/%Y"]
  for date_format in date_formats:
      try:
          datetime.strptime(date_time_str, date_format)
          return True
      except ValueError:
          continue
  return False

# Function to check if a string matches any of the invoice formats
def is_valid_invoice(invoice_str):
  # Define regular expressions for the invoice formats
  invoice_patterns = [
    r'^[A-Z]+-\d{4}-\d{5}$',         # Format 1: SBCC-2324-02446
    r'^[A-Z]{2}-\d{4}-\d{5}$',      # Format 2: EN-2324-03793
    r'^[A-Z]\d+/\d{2}-\d{2}$'       # Format 3: S03677/23-24
    ]
  for pattern in invoice_patterns:
      if re.match(pattern, invoice_str):
          return True
  return False

def supplier_name(input_string):
  if "BHAGYALAKSHM" in input_string:
    return "SRI BHAGYALAKSHMI ENTERPRISES"

def capture_tax_info():
  boxes1 = []
  texts1 = []
  probabilities1 = []
  flag = False
  for c, line in enumerate(output):
    if "Tax Invoice" in line[1][0]:
      flag = True
    if flag == True:
      boxes1.append(line[0])
      texts1.append(line[1][0])
      probabilities1.append(line[1][1])
    if "S.No" in line[1][0]:
      flag = False
  invoice_number = ""
  invoice_date = ""
  GST_Number = ""
  Supplier_Name = ""
  GST_flag = True
  for line in texts1:

    if line.startswith("GSTIN"):
      tt = line.split(":")
      tt = tt[-1]
      print(tt)
      if GST_flag == True:
        if is_valid_gstin(tt):
          GST_Number = tt
          GST_flag = False
    elif GST_flag == True:
        if is_valid_gstin(line):
          GST_Number = line
    if is_valid_invoice(line):
      invoice_number = line
    if is_valid_date_time(line):
      invoice_date = line
    if "BHAGYALAKSHM" in line:
      Supplier_Name = "SRI BHAGYALAKSHMI ENTERPRISES"

  return invoice_number,invoice_date,GST_Number,Supplier_Name

def capture_product_info():
  boxes = []
  texts = []
  probabilities = []
  flag = False
  tcs_flag = False
  TCS = ""
  for c, line in enumerate(output):
    if "S.No" in line[1][0]:
      flag = True
    if flag == True:
      boxes.append(line[0])
      texts.append(line[1][0])
      probabilities.append(line[1][1])
    if "Final Total" in line[1][0]:
      flag = False
    if tcs_flag == True:
      TCS = line[1][0]
      tcs_flag = False
    if "TCS" in line[1][0]:
      tcs_flag = True
  return TCS,boxes,texts,probabilities

In [4]:
def intersection(box_1, box_2):
  return [box_2[0], box_1[1],box_2[2], box_1[3]]

def iou(box_1, box_2):

  x_1 = max(box_1[0], box_2[0])
  y_1 = max(box_1[1], box_2[1])
  x_2 = min(box_1[2], box_2[2])
  y_2 = min(box_1[3], box_2[3])

  inter = abs(max((x_2 - x_1, 0)) * max((y_2 - y_1), 0))
  if inter == 0:
      return 0

  box_1_area = abs((box_1[2] - box_1[0]) * (box_1[3] - box_1[1]))
  box_2_area = abs((box_2[2] - box_2[0]) * (box_2[3] - box_2[1]))

  return inter / float(box_1_area + box_2_area - inter)

def draw_horizonatl_lines(cv2,image_cv,invoice_number,invoice_date,GST_Number,Supplier_Name,tcs,ext_name,boxes,texts,probabilities,output_filename):
  image_boxes = image_cv.copy()

  for box,text in zip(boxes,texts):
    cv2.rectangle(image_boxes, (int(box[0][0]),int(box[0][1])), (int(box[2][0]),int(box[2][1])),(0,0,255),1)
    cv2.putText(image_boxes, text,(int(box[0][0]),int(box[0][1])),cv2.FONT_HERSHEY_SIMPLEX,1,(222,0,0),1)

  cv2.imwrite("detections"+ext_name+".jpg", image_boxes)

  im = image_cv.copy()

  horiz_boxes = []
  vert_boxes = []

  for box in boxes:
    x_h, x_v = 0,int(box[0][0])
    y_h, y_v = int(box[0][1]),0
    width_h,width_v = image_width, int(box[2][0]-box[0][0])
    height_h,height_v = int(box[2][1]-box[0][1]),image_height

    horiz_boxes.append([x_h,y_h,x_h+width_h,y_h+height_h])
    vert_boxes.append([x_v,y_v,x_v+width_v,y_v+height_v])

    cv2.rectangle(im,(x_h,y_h), (x_h+width_h,y_h+height_h),(0,0,255),1)
    cv2.rectangle(im,(x_v,y_v), (x_v+width_v,y_v+height_v),(0,255,0),1)
  #horiz_boxes = tf.squeeze(horiz_boxes)
  #score_threshold = tf.squeeze(score_threshold)
  print(horiz_boxes)
  cv2.imwrite("horiz_vert"+ext_name+".jpg",im)
  print("before")
  """
  SCRIPT IS FAILING "horiz_boxes" VARIABLE RETURNING NULL VALUES AS IT IS NOT ABLE TO DRAW VERTICAL LINES TO THE horiz_vertSample71.pdf.jpg FILE
  """
  horiz_out = tf.image.non_max_suppression(
    horiz_boxes,
    probabilities,
    max_output_size = 1000,
    iou_threshold=0.1,
    #score_threshold=float('-inf'),
    name=None
    )
  print("after")
  horiz_lines = np.sort(np.array(horiz_out))

  im_nms = image_cv.copy()

  for val in horiz_lines:
    cv2.rectangle(im_nms, (int(horiz_boxes[val][0]),int(horiz_boxes[val][1])), (int(horiz_boxes[val][2]),int(horiz_boxes[val][3])),(0,0,255),1)

  cv2.imwrite("im_nms"+ext_name+".jpg",im_nms)
  vert_boxes = tf.squeeze(vert_boxes)
  vert_out = tf.image.non_max_suppression(
    vert_boxes,
    probabilities,
    max_output_size = 1000,
    iou_threshold=0.1,
    #score_threshold=float('-inf'),
    name=None
    )

  vert_lines = np.sort(np.array(vert_out))

  for val in vert_lines:
    cv2.rectangle(im_nms, (int(vert_boxes[val][0]),int(vert_boxes[val][1])), (int(vert_boxes[val][2]),int(vert_boxes[val][3])),(255,0,0),1)

  cv2.imwrite("im_nms"+ext_name+".jpg",im_nms)

  out_array = [["" for i in range(len(vert_lines))] for j in range(len(horiz_lines))]

  unordered_boxes = []

  for i in vert_lines:
    #print(vert_boxes[i])
    unordered_boxes.append(vert_boxes[i][0])

  ordered_boxes = np.argsort(unordered_boxes)

  for i in range(len(horiz_lines)):
    for j in range(len(vert_lines)):
      resultant = intersection(horiz_boxes[horiz_lines[i]], vert_boxes[vert_lines[ordered_boxes[j]]] )

      for b in range(len(boxes)):
        the_box = [boxes[b][0][0],boxes[b][0][1],boxes[b][2][0],boxes[b][2][1]]
        if(iou(resultant,the_box)>0.1):
          out_array[i][j] = texts[b]
  indices = [index for index, item in enumerate(out_array) if "Description of Goods" in item]

  indices_start = [index for index, item in enumerate(out_array) if "PRODUCT DESCRIPTION" in item or "Description of Goods" in item]
  indices_end = [index for index, item in enumerate(out_array) if "Total" in item or "Totai" in item]
  #indices_end = np.where(out_array == "Total")[0]
  print(out_array)
  print(indices_start)
  print(indices_end)
  out_array1=np.array(out_array[indices_start[0]:indices_end[0]])


  df1 = pd.DataFrame(data = out_array1[1:, 1:],index = out_array1[1:, 0],columns = out_array1[0, 1:])
  df1 = df1.loc[:, df1.columns != '']
  df1.columns = ['PRODUCT DESCRIPTION', 'HSN/SAC', 'Qty.', 'Unit Rate', 'GST (%)', 'GST Amt', 'Amount']
  df1.replace('', np.nan, inplace=True)
  df1 = df1.dropna(how='all')
  df1 = df1[df1['HSN/SAC'] != 'SAC']
  df1.insert(0, 'GST_Number', GST_Number)
  df1.insert(0, 'invoice_date', invoice_date)
  df1.insert(0, 'invoice_number', invoice_number)
  df1.insert(0, 'Supplier_Name', Supplier_Name)
  #df1['TCS'] = [''] * (len(df1)-1) + [tcs]
  df1['TCS'] = None
  if tcs is not None:
    df1.at[df1.index[-1], 'TCS'] = tcs
  else:
    df1.at[df1.index[-1], 'TCS'] = None

  file_path = Path(output_filename)
  if file_path.exists():
    df1.to_csv(file_path, index = False, header=False, mode='a')
  else:
    df1.to_csv(file_path, index = False, header=True, mode='w')

  #df1.to_csv('sample3d2.csv',index = False)

In [5]:
directory_path = "/content/data"  # Change this to the path of your directory
output_filename = "/content/sample_15102023.csv"

# List all files in the directory
file_list = glob.glob(os.path.join(directory_path, "*"))

# Iterate through the list of files
for file_path in file_list:
  images = convert_from_path(file_path)
  !mkdir pages

  split_result = file_path.split("/")
  print(split_result[-1])
  print(type(split_result[-1]))
  for i in range(len(images)):
    images[i].save("pages/page"+split_result[-1]+str(i)+".jpg", "JPEG")
  image = cv2.imread("/content/pages/page"+split_result[-1]+str(0)+".jpg")

  image = image[..., ::-1]

  # load model
  model = lp.PaddleDetectionLayoutModel(config_path="lp://PubLayNet/ppyolov2_r50vd_dcn_365e_publaynet/config",
                                  threshold=0.5,
                                  label_map={0:"Text", 1:"Title", 2:"List", 3:"Table", 4:"Figure"},
                                  enforce_cpu=False,
                                  enable_mkldnn=True)#math kernel library
  # detect
  layout = model.detect(image)
  x_1=0
  y_1=0
  x_2=0
  y_2=0
  for l in layout:
    #print(l)
    if l.type == 'Figure':
      x_1 = int(l.block.x_1)
      #print(l.block.x_1)
      y_1 = int(l.block.y_1)
      x_2 = int(l.block.x_2)
      y_2 = int(l.block.y_2)

      break
  im = cv2.imread("/content/pages/page"+split_result[-1]+str(0)+".jpg")
  cv2.imwrite("ext_im"+split_result[-1]+".jpg", im[y_1:y_2,x_1:x_2])
  ocr = PaddleOCR(lang='en')
  image_path = "/content/ext_im"+split_result[-1]+".jpg"
  image_cv = cv2.imread(image_path)
  image_height = image_cv.shape[0]
  image_width = image_cv.shape[1]
  output = ocr.ocr(image_path)[0]
  invoice_number,invoice_date,GST_Number,Supplier_Name = capture_tax_info()
  tcs,boxes,texts,probabilities = capture_product_info()
  draw_horizonatl_lines(cv2,image_cv,invoice_number,invoice_date,GST_Number,Supplier_Name,tcs,split_result[-1],boxes,texts,probabilities,output_filename)

Sample71.pdf
<class 'str'>
download https://paddle-model-ecology.bj.bcebos.com/model/layout-parser/ppyolov2_r50vd_dcn_365e_publaynet.tar to /root/.paddledet/inference_model/ppyolov2_r50vd_dcn_365e_publaynet/ppyolov2_r50vd_dcn_365e_publaynet_infer/ppyolov2_r50vd_dcn_365e_publaynet.tar


100%|██████████| 221M/221M [00:29<00:00, 7.60MiB/s]


download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 4.00M/4.00M [00:14<00:00, 276kiB/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


...

download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:13<00:00, 161kiB/s]

[2023/12/04 16:08:06] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_path='

[2023/12/04 16:08:07] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2023/12/04 16:08:08] ppocr DEBUG: dt_boxes num : 177, elapsed : 1.0256547927856445
[2023/12/04 16:08:37] ppocr DEBUG: rec_res num  : 177, elapsed : 29.151111125946045
29AAPPN7598F1ZS
29ACDPN2062N1ZA
[]
before


InvalidArgumentError: ignored